In [4]:
data_path = "/home/viethoang/petproject/20202/BigData101/data/2015flight.csv"

In [1]:
from pyspark.sql import SparkSession

In [2]:
sc = SparkSession.builder.master("local").appName("dfex").getOrCreate()

In [3]:
sc

### 1. Read file to dataframe
For faster access of data, use df.cache() to put the data in memory When to use caching: As suggested in this post, it is recommended to use caching in the following situations:

* RDD re-use in iterative machine learning applications
* RDD re-use in standalone Spark applications
* When RDD computation is expensive, caching can help in reducing the cost of recovery in the case one executor fails
##### df.cache() is lazy operation, it does not cache the data until you use it!

In [5]:
df = sc.read.format('csv').load(data_path)

In [6]:
df.count()

257

In [7]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [10]:
# use option to read header
df = sc.read.format('csv').option('header', True).option('inferSchema', True).load(data_path)
df.show(3)
df.printSchema()  # now count is in IntegerType!

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [11]:
# cache the dataframe
df.cache() # cache it on memory if the table will be access frequently
df.createOrReplaceTempView('dfTable')  # this is for running the sql code on it

### 2. Load file with manual schema
* Control data type 
* Avoid precision issue

In [12]:
# df have a schema attribute
print(df.schema)

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,IntegerType,true)))


In [17]:
# specify a schema manually - e.g. what if count is long rather than integer
from pyspark.sql.types import StructType, StructField, StringType, LongType

# field name, data type, nullable
field_1 = StructField('DEST_COUNTRY_NAME', StringType(), True)
field_2 = StructField('ORIGIN_COUNTRY_NAME', StringType(), False)
field_3 = StructField('count', LongType(), False)

manualSchema = StructType([field_1, field_2, field_3])

In [18]:
df = sc.read.format('csv').option('header', True).schema(manualSchema).load(data_path)
df.show(3) 

# now count is long! 
# but nullable is true
# this is because CSV format doesn't provide any tools which allow you to specify data constraints 
# so by definition reader cannot assume that input is not null and your data indeed contains nulls.
df.printSchema()
#csv không cho phép ràng buộc dữ liệu 

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)




### 3. Column, Row, and Create DataFrame from Scratch 
col can be used in expression for select() method.

row is data records.

In [23]:
from pyspark.sql import Row
from pyspark.sql.functions import col
# create a DataFrame from Scratch
# field name, data type, nullable
field_1 = StructField('DEST_COUNTRY_NAME', StringType(), True)
field_2 = StructField('ORIGIN_COUNTRY_NAME', StringType(), True)
field_3 = StructField('count', LongType(), False)

manualSchema = StructType([field_1, field_2, field_3])

newRow = Row('NYC', 'MIA', 2)

newDF = sc.createDataFrame([newRow], manualSchema)
newDF.show()
#cơ bản thì giống pandas có thêm 2 cái class col,row

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|              NYC|                MIA|    2|
+-----------------+-------------------+-----+




### 4. Select & SelectExpr 
Flexible expression on columns data.

* use expr() in methods such as select() and withColumn()
* use selectExpr() instead of select(expr(..), expr(..), ..)

In [24]:
df.show(4)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
+-----------------+-------------------+-----+
only showing top 4 rows



In [32]:
df.select("ORIGIN_COUNTRY_NAME").show(4)

+-------------------+
|ORIGIN_COUNTRY_NAME|
+-------------------+
|            Romania|
|            Croatia|
|            Ireland|
|      United States|
+-------------------+
only showing top 4 rows



In [33]:
from pyspark.sql.functions import expr

# use AS to rename the column name
# the returned new DataFrame will have a new column name
newdf = df.select(expr("DEST_COUNTRY_NAME as destination"), 
          expr('ORIGIN_COUNTRY_NAME as departure'))
newdf.show(4)

+-------------+-------------+
|  destination|    departure|
+-------------+-------------+
|United States|      Romania|
|United States|      Croatia|
|United States|      Ireland|
|        Egypt|United States|
+-------------+-------------+
only showing top 4 rows



In [34]:
# use * to select all column
# expr can take some more operations between columns to create new column
newdf.select('*', expr('destination=departure as invalid')).show(4)


+-------------+-------------+-------+
|  destination|    departure|invalid|
+-------------+-------------+-------+
|United States|      Romania|  false|
|United States|      Croatia|  false|
|United States|      Ireland|  false|
|        Egypt|United States|  false|
+-------------+-------------+-------+
only showing top 4 rows



In [35]:
newdf.selectExpr('*', 'destination=departure as invalid','1 as two','false as true').show(4)


+-------------+-------------+-------+---+-----+
|  destination|    departure|invalid|two| true|
+-------------+-------------+-------+---+-----+
|United States|      Romania|  false|  1|false|
|United States|      Croatia|  false|  1|false|
|United States|      Ireland|  false|  1|false|
|        Egypt|United States|  false|  1|false|
+-------------+-------------+-------+---+-----+
only showing top 4 rows



### 5. Column Manipulation 
For maniplating column

* use df.withColumn() to add column, change column type.
 * this method takes two params: column name, expression
 * if column name does not exist, it will append a new column
 * if column name exist, it will replace the column with new expression result
* use df.withColumnRenamed() to rename a column
 * column name should avoid reserved characters and keywords such as as. If needed, use `...` to skip.
 * use df.drop() to drop a column
 * Spark session is by default case insensitive.

* use spark_session.sql('set spark.sql.caseSensitive=false') to change to case sensitive
* Add an id column:

 * monotonically_increasing_id() return an id column


### 6. Row Manipulation 
Basic Manipluation including:

* Filtering: filter rows with some expression with T/F output: df.filter() or df.where()
 * note that df.select(condition) will return a table with single column of true/false.
 * Get Unique: df.distinct()
 * Random Samples: df.sample(withReplacement=, fraction=, seed=)
 * Random Splits: df.randomSplit(fractions=, seed=)
 * Concat and Append: df.union(
 * Sort: df.orderBy(expr or col or col_name)
 * Limit: df.limit(n)